In [71]:
import utils
import sagemaker
import json
import mlflow

In [58]:
# Detecting Execution Role
sagemaker.get_execution_role()

'arn:aws:iam::635106763104:role/service-role/SageMaker-ExecutionRole-20250524T225979'

# Loading and saving the MLFlow model locally

In [ ]:
# Set tracking server
tracking_server_arn = 'arn:aws:sagemaker:us-east-1:654654589924:mlflow-tracking-server/mlops-utec-mlflow-server'
mlflow.set_tracking_uri(tracking_server_arn)

In [ ]:
# Load model
model_name = "credit-card-fraud-detection"
model_version = "latest"
model_uri = f"models:/{model_name}/{model_version}"
model = mlflow.xgboost.load_model(model_uri)

In [ ]:
# Save model locally
mlflow.xgboost.save_model(model,"model")

# Checking current images

In [2]:
!docker images

REPOSITORY                                                                                                TAG       IMAGE ID       CREATED             SIZE
635106763104.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi   latest    94ccfda130af   About an hour ago   2.43GB
utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi                                                latest    94ccfda130af   About an hour ago   2.43GB


# Creating ECR repository

In [3]:
user = utils.get_username()
repository_name = f"utec-mlops/{user}/online-prediction/api/rest/fastapi"
repository_name

'utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi'

In [6]:
!aws ecr create-repository --repository-name $repository_name

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:635106763104:repository/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi",
        "registryId": "635106763104",
        "repositoryName": "utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi",
        "repositoryUri": "635106763104.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi",
        "createdAt": "2025-06-11T22:38:25.455000+00:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


# Build image

In [163]:
!docker build --network sagemaker --no-cache -t $repository_name .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.

Sending build context to Docker daemon  200.2kB
Step 1/8 : FROM python:3.8
 ---> 3ea6eaad4f17
Step 2/8 : WORKDIR /app
 ---> Running in 98bbaf8672d9
 ---> Removed intermediate container 98bbaf8672d9
 ---> 0504a76ea731
Step 3/8 : COPY model /app/model
 ---> 279631ffff8b
Step 4/8 : COPY app.py requirements.txt /app/
 ---> 519dba380577
Step 5/8 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Running in 7b32b996efbf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 216.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 196.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 221.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 175.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Checking current images

In [164]:
!docker images

REPOSITORY                                                   TAG       IMAGE ID       CREATED         SIZE
utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi   latest    94ccfda130af   7 seconds ago   2.43GB


# Login to ECR

In [ ]:
ecr_url = "654654589924.dkr.ecr.us-east-1.amazonaws.com"

In [166]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin $ecr_url

WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores

Login Succeeded


# Tag image

In [44]:
tag_name = f"{repository_name}:latest"
full_tag_name = f"{ecr_url}/{tag_name}"
print(tag_name)
print(full_tag_name)

utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi:latest
635106763104.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi:latest


In [168]:
!docker tag $tag_name $full_tag_name

# Checking current images

In [169]:
!docker images

REPOSITORY                                                                                                TAG       IMAGE ID       CREATED          SIZE
635106763104.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi   latest    94ccfda130af   18 seconds ago   2.43GB
utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi                                                latest    94ccfda130af   18 seconds ago   2.43GB


# Push image to ECR

In [170]:
!docker push $full_tag_name

The push refers to repository [635106763104.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi]

19f50cc0: Preparing 
939e7de8: Preparing 
1c71f6cb: Preparing 
30ccc3fe: Preparing 
710ca3c7: Preparing 
e4d52b5a: Preparing 
8afd69b3: Preparing 
433c3a29: Preparing 
c7a486d9: Preparing 
a6961052: Preparing 
latest: digest: sha256:a1271b0f5224ecf2d7f29c74b251114f4f837f254ea24cf18b0c2aa9c217b94c size: 2630


# Create App Runner Service

In [64]:
service_name = f"utec-mlops-{user}-fastapi"
app_runner_role = "arn:aws:iam::635106763104:role/service-role/AppRunnerECRAccessRole"

In [68]:
conf = {
    "ServiceName": service_name,
    "SourceConfiguration": {
        "AuthenticationConfiguration": {
            "AccessRoleArn": app_runner_role
        },
        "ImageRepository": {
            "ImageIdentifier": full_tag_name,
            "ImageConfiguration": {
                "Port": "8000"
            },
            "ImageRepositoryType": "ECR"
        }
    },
    "InstanceConfiguration": {
        "Cpu": "2 vCPU",
        "Memory": "4 GB"
    }
}

In [69]:
with open('app_runner_config.json', 'w') as f:
    json.dump(conf, f)

In [70]:
!aws apprunner create-service --cli-input-json file://app_runner_config.json

{
    "Service": {
        "ServiceName": "utec-mlops-ricardo-puma-fastapi",
        "ServiceId": "60ef59b9affe4b759b94407ddc974ddf",
        "ServiceArn": "arn:aws:apprunner:us-east-1:635106763104:service/utec-mlops-ricardo-puma-fastapi/60ef59b9affe4b759b94407ddc974ddf",
        "ServiceUrl": "c5ar3bmsn6.us-east-1.awsapprunner.com",
        "CreatedAt": "2025-06-12T07:10:18.847000+00:00",
        "UpdatedAt": "2025-06-12T07:10:18.847000+00:00",
        "Status": "OPERATION_IN_PROGRESS",
        "SourceConfiguration": {
            "ImageRepository": {
                "ImageIdentifier": "635106763104.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi:latest",
                "ImageConfiguration": {
                    "Port": "8000"
                },
                "ImageRepositoryType": "ECR"
            },
            "AutoDeploymentsEnabled": false,
            "AuthenticationConfiguration": {
                "AccessRoleArn": "arn:aws:iam::6

# Testing the Model API

In [72]:
!curl -X POST "<Default domain>/predict/" \
  -H "Content-Type: application/json" \
  -d '{"features": [1,2,3,4,5]}'

{"prediction":0.0018256510375067592}

In [ ]:
# 1)
# Try it out in https://reqbin.com/ 
# with the body:
# {"features": [1,2,3,4,5]}

In [ ]:
# 2)
# Download the file "card_fraud_detection_portal.html"
# replace the API domain in the file and open the file in your browser 

# Delete Image

In [ ]:
!docker rmi <IMAGE_ID>   
!docker rmi <FULL_TAG_NAME>

In [162]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE
